In [15]:
# Importa as bibliotecas necessárias
import pandas as pd
import datetime
import re
import os
import lxml
import html5lib
from bs4 import BeautifulSoup
import requests

In [8]:
# Define a localização de arquivos e variáveis importantes
HISTORICO = '../dados/corona_brasil.csv'
LINK_MINSAUDE = '../dados/auxiliares/link_minsaude.csv'

meses = {
  "Janeiro": "01",
  "Janeiro": "02",
  "Março": "03",
  "Abril": "04",
  "Maio": "05",
}
HOJE = str(datetime.date.today())

HOJE

'2020-03-28'

In [12]:
# Lê a tabela com links do Ministério da Saúde e encontra o dia de hoje
acervo = pd.read_csv(LINK_MINSAUDE)

url = acervo[acervo.Data == HOJE].Link.values[0]


In [13]:
# Baixa o link 
arquivo = requests.get(url).text

In [17]:
# Extração do dia a partir do texto da notícia
soup = BeautifulSoup(arquivo, 'lxml') 

ultima_modificacao = soup.find('span', {'class' : 'documentModified'}).text

timedate = re.search("[0-9][^\\t]*", ultima_modificacao).group().split(",")[0].split(" de ")

dia = timedate[0]
mes = meses[timedate[1]]
ano = timedate[2]

DIA = ano + '-' + mes + '-' + dia

DIA

'2020-03-28'

In [18]:
# Lê o arquivo HTML com os casos por estado

table = soup.find_all('table')[0] 

rows = table.find_all('tr')

df = []

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    df.append([ele for ele in cols if ele])
    
df = pd.DataFrame(df)

# Define nome das colunas
df.columns = df.iloc[0]

# Retira colunas desnecessárias para a conciliação da série histórica
df = df.iloc[2:,1:4]

df

,UF/REGIÃO,CONFIRMADOS,ÓBITOS
2,"184 (4,7%)",1,"0,5%"
3,AC,25,-
4,AM,111,1
5,AP,4,-
6,PA,17,-
7,RO,6,-
8,RR,12,-
9,TO,9,-
10,"624 (16,0%)",7,"1,1%"
11,AL,14,-


In [19]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # Remove entradas impróprias
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    df = df[df.CONFIRMADOS.notnull()]
# Remove valores percentuais
df = df[~df['ÓBITOS'].str.contains('%')] 
# Tira o ponto como separador decimal
df.CONFIRMADOS = df.CONFIRMADOS.str.replace('.', '') 

df

,UF/REGIÃO,CONFIRMADOS,ÓBITOS
3,AC,25,-
4,AM,111,1
5,AP,4,-
6,PA,17,-
7,RO,6,-
8,RR,12,-
9,TO,9,-
11,AL,14,-
12,BA,128,-
13,CE,314,4


In [24]:
# Adiciona coluna de data e renomeia
df['date'] = DIA

df.rename(columns={'UF/REGIÃO':'uf', 'ÓBITOS':'deaths', 'CONFIRMADOS':'cases'}, inplace=True)

In [22]:
# Corrige o campo de óbitos
df.deaths = df.deaths.str.replace('-', '') 

df

,uf,cases,deaths,date
3,AC,25,,2020-03-28
4,AM,111,1,2020-03-28
5,AP,4,,2020-03-28
6,PA,17,,2020-03-28
7,RO,6,,2020-03-28
8,RR,12,,2020-03-28
9,TO,9,,2020-03-28
11,AL,14,,2020-03-28
12,BA,128,,2020-03-28
13,CE,314,4,2020-03-28


In [27]:
# Carrega a série histórica e junta com o dia atual, removendo duplicadas
final = pd.read_csv(HISTORICO).append(df,ignore_index=True)

# TO DO: Remover entradas duplicadas com o drop_duplicates()
final

,uid,date,time,suspects,refuses,confirmado,deads,local,cases,comments,broadcast,deaths,uf,index
0,11.0,2020-03-25,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,0,RO,NaN
1,12.0,2020-03-25,NaN,NaN,NaN,NaN,NaN,NaN,23,NaN,NaN,0,AC,NaN
2,13.0,2020-03-25,NaN,NaN,NaN,NaN,NaN,NaN,54,NaN,NaN,1,AM,NaN
3,14.0,2020-03-25,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,0,RR,NaN
4,15.0,2020-03-25,NaN,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,0,PA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,NaN,2020-03-28,NaN,NaN,NaN,NaN,NaN,NaN,31,NaN,NaN,,MS,NaN
935,NaN,2020-03-28,NaN,NaN,NaN,NaN,NaN,NaN,13,NaN,NaN,,MT,NaN
936,NaN,2020-03-28,NaN,NaN,NaN,NaN,NaN,NaN,133,NaN,NaN,2,PR,NaN
937,NaN,2020-03-28,NaN,NaN,NaN,NaN,NaN,NaN,184,NaN,NaN,1,SC,NaN


In [28]:
# Exporta o CSV
final.to_csv("../dados/corona_brasil.csv", index = False)